In [1]:
import numpy as np
import pandas as pd
import polars as pl
import pytomlpp as toml
from data_preprocessing import getXY, data_normalizeation
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from tqdm import tqdm

In [2]:
# read config
cfg = toml.load('/home/gyli/GP_HP/cfg/feature_selection.toml')
data_path = cfg['data_path']
variance_filter_thres = cfg['variance_filter_thres']

In [3]:
raw_x, raw_y = getXY(data_path)
# raw_x = data_normalizeation(raw_x)
future_names = raw_x.columns
pd_x, pd_y = raw_x.to_pandas(), raw_y.to_pandas()

In [16]:
raw_x.describe()

statistic,年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",52.254056,4.567131,2.090111,0.170863,0.017453,70.440742,43.618026,26.823058,11.517256,3.229566,4.082569,141.472249,2.334196,5.272611,1.085323,1.977849,1.291036,2.957736,91.495374,12.660115,0.998116,32.570699,3.596984
"""std""",15.386269,1.994131,0.664631,0.141238,0.019557,6.839072,5.926141,3.912165,5.942347,2.291735,0.37868,2.276023,0.131797,2.742084,0.555307,1.602298,0.321843,0.878035,5.036598,1.284763,0.117857,3.618886,0.549035
"""min""",1.0,0.05,0.13,0.01,0.000287,27.3,11.6,11.6,0.8,0.1,2.1,106.4,1.29,0.2,0.1,0.07,0.149165,0.01,57.6,5.198928,0.297887,13.710337,0.81
"""25%""",41.0,3.519042,1.85,0.11,0.01,67.0,40.7,24.3,8.0,2.3,3.914825,140.6,2.274896,4.0,0.8,1.13,1.09,2.42,90.796167,12.4,0.969918,31.946807,3.408363
"""50%""",52.0,4.11,2.117365,0.164643,0.012574,71.4,44.8,26.6,10.6,2.993746,4.083333,141.771598,2.35,5.0,1.0,1.695664,1.28,2.929331,91.958932,12.71707,0.996111,32.902395,3.587174
"""75%""",64.0,4.98,2.23,0.181875,0.02,74.9,47.7,29.0,13.5,3.7,4.2,142.5,2.41,5.8,1.3,2.2,1.431957,3.39,93.117697,13.022838,1.023449,33.735237,3.7556
"""max""",99.0,51.36,52.5,6.35,1.6,102.6,59.6,66.8,236.3,144.3,19.5,164.6,4.37,63.3,19.0,35.59,4.73,15.13,133.6,37.1,3.79,97.9,11.04


In [33]:
raw_x = pl.read_csv('/home/gyli/GP_HP/data/hypertension.csv', dtypes={'年龄' : pl.Float64})

In [34]:
a = raw_x.describe()
for c in a.columns[1:]:
    a = a.with_columns(pl.col(c).round(3))
a.to_pandas().transpose().to_csv('/home/gyli/GP_HP/result/data_clean_describe.csv')

In [37]:
df = raw_x.to_pandas()

In [38]:
pd[pd.isnull().any(axis=1)]

TypeError: isna() missing 1 required positional argument: 'obj'

In [4]:
svm_clf = SVC()
X, Y = raw_x.to_pandas(), np.ravel(raw_y.to_pandas())

In [6]:
# filter by variance threshold
# from sklearn.feature_selection import VarianceThreshold
# var_thres = VarianceThreshold(threshold=variance_filter_thres)
# var_thres.fit(X)
# var_thres_col_idx = var_thres.get_support(indices=True)
# X_Var = X.iloc[:, var_thres_col_idx]
# print("\033[34m使用方差阈值筛选出来的特征结果索引:",var_thres.get_support(indices=True))
# print("\033[34m方差阈值选择前的结果:",cross_val_score(estimator=svm_clf,X=X,y=Y,cv=5,scoring='accuracy').mean())
# print("\033[34m方差阈值选择后的结果:",cross_val_score(estimator=svm_clf,X=X_Var,y=Y,cv=5,scoring='accuracy').mean())
X_Var = X

In [7]:
# 归一化
# X_Var = data_normalizeation(pl.from_pandas(X_Var))

In [8]:
list(X.columns)

['年龄',
 '中性粒细胞数',
 '淋巴细胞数',
 '嗜酸性粒细胞数',
 '嗜碱性粒细胞数',
 '总蛋白',
 '白蛋白',
 '球蛋白',
 '总胆红素',
 '直接胆红素',
 '钾',
 '钠',
 '钙',
 '尿素氮',
 '谷草谷丙',
 '甘油三酯',
 '高密度脂蛋白',
 '低密度脂蛋白',
 '红细胞平均体积',
 '凝血酶原时间',
 '国际标准化比值',
 '活化部分凝血活酶时间',
 '纤维蛋白原']

In [9]:
# filter by Chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
chi2_feature = SelectKBest(score_func=chi2, k='all')
chi2_selector = chi2_feature.fit(X_Var, Y)
chi2_selector.scores_
chi2_df = pd.DataFrame(np.arange(len(chi2_selector.scores_)), columns=['feature_idx'])
chi2_df['feature'] = list(X_Var.columns)
chi2_df['chi2_value'] = chi2_selector.scores_
sort_chi2 = chi2_df.sort_values(by='chi2_value', ascending=False).reset_index(drop=True)
print(sort_chi2)
X_Chi2_top10 = X_Var.iloc[:, sort_chi2[:10]['feature_idx'].to_list()]
X_Chi2_top14 = X_Var.iloc[:, sort_chi2[:14]['feature_idx'].to_list()]
print("\033[34m卡方选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Var,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m卡方选择top14后的结果:",cross_val_score(estimator=svm_clf,X=X_Chi2_top10,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m卡方选择top10后的结果:",cross_val_score(estimator=svm_clf,X=X_Chi2_top14,y=Y,cv=5,scoring='accuracy').mean())

    feature_idx     feature   chi2_value
0             6         白蛋白  3607.793828
1             5         总蛋白  2232.444816
2             0          年龄  1620.557003
3             1      中性粒细胞数  1262.392571
4            21  活化部分凝血活酶时间   383.776717
5            14        谷草谷丙   246.632228
6            15        甘油三酯   196.700687
7             8        总胆红素   102.868027
8             9       直接胆红素    90.602259
9             2       淋巴细胞数    89.572173
10           17      低密度脂蛋白    48.415306
11           18     红细胞平均体积    43.963457
12           19      凝血酶原时间    43.712427
13           11           钠    41.472782
14           13         尿素氮    22.948253
15           12           钙    17.311755
16            3     嗜酸性粒细胞数    15.833180
17           10           钾    14.112563
18            4     嗜碱性粒细胞数    10.461442
19           22       纤维蛋白原     3.729141
20           16      高密度脂蛋白     0.603448
21           20     国际标准化比值     0.559221
22            7         球蛋白     0.000358
卡方选择前的结果: 0.7198

In [10]:
X_Chi2 = X_Var.iloc[:, sort_chi2[:14]['feature_idx'].to_list()]

In [11]:
# 相关系数筛选
corr = X_Chi2.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool_))
to_drop = [col for col in upper.columns if any(upper[col] > 0.8)]
print('deselect : ', to_drop)
upper

deselect :  ['总蛋白', '直接胆红素']


,白蛋白,总蛋白,年龄,中性粒细胞数,活化部分凝血活酶时间,谷草谷丙,甘油三酯,总胆红素,直接胆红素,淋巴细胞数,低密度脂蛋白,红细胞平均体积,凝血酶原时间,钠
白蛋白,NaN,0.821454,0.061810,0.460003,0.188782,0.334556,0.006366,0.211114,0.020222,0.214614,0.046843,0.077524,0.120331,0.356134
总蛋白,NaN,NaN,0.129596,0.338535,0.166047,0.256663,0.048175,0.153805,0.030848,0.196337,0.095966,0.024375,0.097896,0.244804
年龄,NaN,NaN,NaN,0.177365,0.131641,0.028208,0.138757,0.092833,0.104352,0.051793,0.075182,0.095411,0.169505,0.121082
中性粒细胞数,NaN,NaN,NaN,NaN,0.093932,0.147988,0.061344,0.111389,0.025697,0.038299,0.002683,0.054687,0.058327,0.351828
活化部分凝血活酶时间,NaN,NaN,NaN,NaN,NaN,0.117652,0.150532,0.066922,0.032863,0.031075,0.061671,0.174340,0.740851,0.126529
谷草谷丙,NaN,NaN,NaN,NaN,NaN,NaN,0.086485,0.109115,0.038425,0.141833,0.036579,0.007635,0.077541,0.122886
甘油三酯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.085004,0.085252,0.064004,0.024384,0.064693,0.217243,0.092386
总胆红素,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.839410,0.006889,0.016982,0.049823,0.088564,0.004142
直接胆红素,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059185,0.083656,0.022396,0.070193,0.070869
淋巴细胞数,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.080059,0.025527,0.004276,0.131018


In [12]:
X_Corr = X_Chi2.drop(to_drop, axis=1)
print("Corr选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Chi2,y=Y,cv=5,scoring='accuracy').mean())
print("Corr选择后的结果:",cross_val_score(estimator=svm_clf,X=X_Corr,y=Y,cv=5,scoring='accuracy').mean())

Corr选择前的结果: 0.7202619607103495
Corr选择后的结果: 0.7186114133067856


In [13]:
# 随机森林筛选
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_Corr,Y)
feature_importance = rf_clf.feature_importances_
print(feature_importance)
feature_importance_df = pd.DataFrame(feature_importance, columns=['importance'])
feature_importance_df['feature'] = list(X_Corr.columns)
feature_importance_df['featrue_idx'] = np.arange(feature_importance_df.shape[0])
feature_importance_df = feature_importance_df.sort_values(ascending=False, by='importance').reset_index(drop=True)
print(feature_importance_df)
X_forest_top1 = X_Corr.iloc[:, feature_importance_df[:1]['featrue_idx'].to_list()]
X_forest_top6 = X_Corr.iloc[:, feature_importance_df[:6]['featrue_idx'].to_list()]
X_forest_top9 = X_Corr.iloc[:, feature_importance_df[:9]['featrue_idx'].to_list()]
X_forest_top11 = X_Corr.iloc[:, feature_importance_df[:11]['featrue_idx'].to_list()]
print("\033[34m随机森林选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Corr,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top1后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top1,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top6后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top6,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top9后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top9,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top11后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top11,y=Y,cv=5,scoring='accuracy').mean())

[0.16557413 0.07605325 0.08014503 0.09427791 0.04604019 0.07380831
 0.06341807 0.08935498 0.0888211  0.06851107 0.07279351 0.08120246]
    importance     feature  featrue_idx
0     0.165574         白蛋白            0
1     0.094278  活化部分凝血活酶时间            3
2     0.089355       淋巴细胞数            7
3     0.088821      低密度脂蛋白            8
4     0.081202           钠           11
5     0.080145      中性粒细胞数            2
6     0.076053          年龄            1
7     0.073808        甘油三酯            5
8     0.072794      凝血酶原时间           10
9     0.068511     红细胞平均体积            9
10    0.063418        总胆红素            6
11    0.046040        谷草谷丙            4
随机森林选择前的结果: 0.7186114133067856
随机森林选择top1后的结果: 0.7138206801654963
随机森林选择top6后的结果: 0.723844705126697
随机森林选择top9后的结果: 0.7241669850658623
随机森林选择top11后的结果: 0.7191347546414176


In [14]:
X_Forest = X_Corr.iloc[:, feature_importance_df[:9]['featrue_idx'].to_list()]

In [15]:
# 支持向量机前向选择
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1 = SFS(estimator=svm_clf,k_features=7,forward=True,floating=False,verbose=2,scoring='accuracy',cv=5)
sfs1.fit(np.array(X_Forest), Y)
# SFS(estimator=SVC(), k_features=5, scoring='accuracy',verbose=2)

/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fi

SequentialFeatureSelector(estimator=SVC(), k_features=(7, 7),
                          scoring='accuracy', verbose=2)

In [16]:
X_SFS1 = X_Forest.iloc[:, list(sfs1.k_feature_idx_)]
print("SFS前向选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Forest,y=Y,cv=5,scoring='accuracy').mean())
print("SFS前向选择后的结果:",cross_val_score(estimator=svm_clf,X=X_SFS1.values,y=Y,cv=5,scoring='accuracy').mean())

SFS前向选择前的结果: 0.7241669850658623
SFS前向选择后的结果: 0.7263003163899591


In [17]:
X_SFS1

,白蛋白,活化部分凝血活酶时间,低密度脂蛋白,钠,中性粒细胞数,甘油三酯,凝血酶原时间
0,39.546489,33.946848,1.670000,137.200000,9.260000,0.570000,13.323282
1,40.100000,26.000000,0.860000,135.800000,2.890000,1.460000,10.400000
2,39.100000,27.100000,3.220000,139.400000,4.680000,1.030000,10.200000
3,40.500000,32.554017,3.620000,146.300000,4.603406,1.160000,12.714961
4,41.200000,33.047475,3.350000,140.200000,4.847728,1.730000,12.533349
...,...,...,...,...,...,...,...
24836,34.700000,37.800000,3.039262,144.000000,6.280000,1.917397,11.700000
24837,36.900000,38.300000,3.970000,141.699474,9.510000,7.100000,12.300000
24838,43.506534,33.649698,3.133671,143.000000,4.857917,1.670000,12.898219
24839,48.300000,33.556394,2.960000,141.998793,3.770986,0.590000,12.946776


In [20]:
# 暂使用X_SFS1训练SVM
gamma = cfg['gamma']
C = cfg['C']
best_score = 0.0
best_params = {'C' : -1.0, 'gamma' : -1.0}
for g in tqdm(gamma, desc='Outer Loop'):
    for c in tqdm(C, desc='Inner Loop'):
        svm = SVC(gamma=g, C=c)
        # svm.fit(X_SFS1, Y)
        scores = cross_val_score(svm, X_SFS1, Y, cv=5, scoring='accuracy')
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_params['C'] = c
            best_params['gamma'] = g
        tqdm.write(f'finish one param part, c = {c}, gamma = {g}, score = {score}')
tqdm.write(f'best param: C = {best_params["C"]}, gamma = {best_params["gamma"]}, score = {best_score}')


Outer Loop:   0%|          | 0/10 [00:00<?, ?it/s]

                                                  
Outer Loop:   0%|          | 0/10 [03:02<?, ?it/s]

finish one param part, c = 1, gamma = 1.0, score = 0.7846273951640266


                                                  
Outer Loop:   0%|          | 0/10 [06:10<?, ?it/s]         

finish one param part, c = 2, gamma = 1.0, score = 0.7899008474117629


...

finish one param part, c = 8, gamma = 10, score = 0.8072878173176108


                                                              
Outer Loop:  90%|█████████ | 9/10 [8:11:34<53:14, 3194.46s/it]

finish one param part, c = 9, gamma = 10, score = 0.8072475596686575


                                                              
Outer Loop: 100%|██████████| 10/10 [8:16:23<00:00, 2978.37s/it]

finish one param part, c = 10, gamma = 10, score = 0.8073280668648033
best param: C = 8, gamma = 5, score = 0.8104289428595781


In [7]:
mb = ['白蛋白', '活化部分凝血活酶时间', '低密度脂蛋白', '钠', '中性粒细胞数', '甘油三酯', '凝血酶原时间']
t_x = X[mb]

In [8]:
t_x

,白蛋白,活化部分凝血活酶时间,低密度脂蛋白,钠,中性粒细胞数,甘油三酯,凝血酶原时间
0,39.546489,33.946848,1.670000,137.200000,9.260000,0.570000,13.323282
1,40.100000,26.000000,0.860000,135.800000,2.890000,1.460000,10.400000
2,39.100000,27.100000,3.220000,139.400000,4.680000,1.030000,10.200000
3,40.500000,32.554017,3.620000,146.300000,4.603406,1.160000,12.714961
4,41.200000,33.047475,3.350000,140.200000,4.847728,1.730000,12.533349
...,...,...,...,...,...,...,...
24836,34.700000,37.800000,3.039262,144.000000,6.280000,1.917397,11.700000
24837,36.900000,38.300000,3.970000,141.699474,9.510000,7.100000,12.300000
24838,43.506534,33.649698,3.133671,143.000000,4.857917,1.670000,12.898219
24839,48.300000,33.556394,2.960000,141.998793,3.770986,0.590000,12.946776


In [15]:
linear_svm = SVC(kernel='linear', gamma=7, C=5)
linear_svm.fit(t_x, Y)

SVC(C=5, gamma=7, kernel='linear')

In [26]:
p_linear_scores = cross_val_score(linear_svm, t_x, Y, cv=5, scoring='accuracy')

In [ ]:
SVC()

In [14]:
linear_scores = cross_val_score(linear_svm, t_x, Y, cv=5, scoring='accuracy')

In [23]:
linear_scores

array([0.69973838, 0.58494364, 0.90801127, 0.82045089, 0.61594203])

In [9]:
svm = SVC(gamma=2, C=5)
scores = cross_val_score(svm, t_x, Y, cv=5, scoring='accuracy')"""  """

In [12]:
scores

array([0.91024351, 0.70128824, 0.84883253, 0.85346216, 0.70873591])

In [17]:
linear_svm.score(t_x, Y)

# # 在具有二元标签指示符的多标签分类案例中
# print(accuracy_score(np.array([[0, 1], [1, 1]]), np.ones((2, 2))))  # 0.5


0.7287951370717765

In [21]:
default_svm = SVC()
default_svm.fit(t_x, Y)
default_svm.score(t_x, Y)

0.7294794895535607

In [20]:
svm.fit(t_x, Y)
svm.score(t_x, Y)

0.9833742602954793

In [15]:
default_svm = SVC()
scores = cross_val_score(default_svm, t_x, Y, cv=5, scoring='accuracy')

In [24]:
scores

array([0.70014087, 0.58333333, 0.90942029, 0.82065217, 0.61795491])

In [ ]:
params = [()]